In [1]:
import requests
import json
import time
import os
import re

In [2]:
# Download paths for different datasets
data_path = '..//data//'
results_cty_path = '..//data//results_cty//'
results_prv_path = '..//data//results_prv//'
results_reg_path = '..//data//results_reg//'

# URL for list of regions and URL of the directory for regional geolocation datasets
regions_url = 'https://eleksyonconfig.gmanetwork.com/gno/microsites/eleksyon2025/results/ref/NO/geolocation_REGION.json'
per_region_info_url = 'https://data2.gmanetwork.com/gno/microsites/eleksyon2025/results/locations/regions/'

# Format for geolocation datasets
per_region_info_fnstart = 'geolocation_'

# Directory URL for election results data
results_url = 'https://e25a-cf.gmanetwork.com/l/'

## Downloading list of regions

In [3]:
# Download list of regions
regions_json_path = os.path.join(data_path,"geolocation_REGION.json")
if not os.path.isfile(regions_json_path):  # If the file was not already downloaded, download.
    with open(regions_json_path,"w") as f:
        r = requests.get(regions_url)
        print(r.status_code)
        f.write(r.text)

## Download administrative subunits per region

In [4]:
# Open downloaded regions file
with open(regions_json_path,"r") as f:
    regions_json = json.load(f)
print(regions_json)

{'REGION': ['BARMM', 'CORDILLERA ADMINISTRATIVE REGION', 'NATIONAL CAPITAL REGION', 'NIR', 'OAV', 'REGION I', 'REGION II', 'REGION III', 'REGION IV-A', 'REGION IV-B', 'REGION V', 'REGION VI', 'REGION VII', 'REGION VIII', 'REGION IX', 'REGION X', 'REGION XI', 'REGION XII', 'REGION XIII'], 'registered_voters': '69773653'}


In [5]:
# Download geolocation data for each region
for reg in regions_json['REGION']:
    fname = f"geolocation_{reg.replace(" ","_")}.json"
    if not os.path.isfile(os.path.join(data_path,fname)):
        with open(os.path.join(data_path,fname),"w") as f:
            r = requests.get(per_region_info_url+fname)
            print(reg,r.status_code)
            f.write(r.text)

## Download election results

In [6]:
# Open downloaded regions file
with open(regions_json_path,"r") as f:
    regions_json = json.load(f)
print(regions_json)

{'REGION': ['BARMM', 'CORDILLERA ADMINISTRATIVE REGION', 'NATIONAL CAPITAL REGION', 'NIR', 'OAV', 'REGION I', 'REGION II', 'REGION III', 'REGION IV-A', 'REGION IV-B', 'REGION V', 'REGION VI', 'REGION VII', 'REGION VIII', 'REGION IX', 'REGION X', 'REGION XI', 'REGION XII', 'REGION XIII'], 'registered_voters': '69773653'}


In [7]:
# Create download function
def download_file(fname,dir_url=results_url,fpath=data_path):
    if not os.path.isfile(os.path.join(fpath,fname)):
        r = requests.get(dir_url+fname)
        if r.status_code != 200:
            print("Error: ",fname)
            return
        with open(os.path.join(fpath,fname),"w") as f:
            f.write(r.text)

In [8]:
# Convert webpage link property to json file link
def link_to_name(link):
    temp = "_".join(link.split("/")[4:-1]).replace("+","_")
    temp2 = re.sub(r'(%\w{4})+','_',temp)
    return temp2 + '.json'

In [9]:
### Download election results data from regional-level > provincial > city-wide
# Iterate over regions
for reg in regions_json['REGION']:
    with open(os.path.join(data_path,f"geolocation_{reg.replace(" ","_")}.json"),"r") as f:
        reg_json = json.load(f)
    name = link_to_name(reg_json['region']['link'])
    download_file(name,fpath=results_reg_path)

    # Iterate over provinces
    for prov in reg_json['region']['provinces']:
        name = link_to_name(prov['link'])
        download_file(name,fpath=results_prv_path)

        # Iterate over cities/municipalities
        for cty in prov['cities']:
            name = link_to_name(cty['link'])
            download_file(name,fpath=results_cty_path)

Error:  BARMM_BASILAN_HADJI_MUHTAMAD.json
Error:  BARMM_LANAO_DEL_SUR_BAYANG.json
Error:  BARMM_LANAO_DEL_SUR_MAGUING.json
Error:  BARMM_LANAO_DEL_SUR_PAGAYAWAN.json
Error:  BARMM_LANAO_DEL_SUR_SULTAN_DUMALONDONG.json
Error:  BARMM_LANAO_DEL_SUR_TAGOLOAN.json
Error:  OAV_EUROPE_RUSSIA.json
Error:  OAV_MIDDLE_EAST_AND_AFRICAS_IRAN.json
Error:  OAV_MIDDLE_EAST_AND_AFRICAS_SYRIA.json
Error:  REGION_IX_SULU_PANGUTARAN.json


## Download national results

In [11]:
if not os.path.isfile(os.path.join(data_path,'SENATOR_OF_PHILIPPINES.json')):
    r = requests.get('https://e25a-cf.gmanetwork.com/n/SENATOR_OF_PHILIPPINES.json')
    print(r.status_code)
    with open(os.path.join(data_path,'SENATOR_OF_PHILIPPINES.json'),"w") as f:
        f.write(r.text)

200
